In [53]:
import pandas as pd
import seaborn
from IPython import embed
import elo_functions as elo
from matplotlib import pyplot as plt
from scipy import optimize
import numpy as np


In [54]:
matches_df = pd.read_excel("../../Data/dartsdatabase/Results2.xlsx")

In [55]:
matches_df.head()

,player_name,date,event,category,event_round,result,opponent,score
0,Michael van Gerwen,10/05/2018,Premier League,PDC U,League 10,Fixture,Gary Anderson,
1,Michael van Gerwen,06/05/2018,PDC European Darts Grand Prix,PDC ET,Final,Won,James Wade,8 V 3
2,Michael van Gerwen,06/05/2018,PDC European Darts Grand Prix,PDC ET,Semi Final,Won,Michael Smith,7 V 0
3,Michael van Gerwen,06/05/2018,PDC European Darts Grand Prix,PDC ET,Quarter Final,Won,Dave Chisnall,6 V 4
4,Michael van Gerwen,06/05/2018,PDC European Darts Grand Prix,PDC ET,Last 16,Won,Darren Webster,6 V 5


In [56]:
def outcome_map(result):
    if result == "Won":
        return 1.0
    elif result == "Lost":
        return 0.0
    else:
        return 0.5

#Drop matches that haven't yet happened
matches_df = matches_df[matches_df['result'] != 'fixture']

#Why are there some dups? Need to investigate
matches_df.drop_duplicates(inplace=True)

#Map outcomes to numerical values
matches_df['Outcome'] = matches_df['result'].apply(outcome_map)

In [68]:
#Rename columns - same as other versions
matches_df.rename(index=str, columns={"player_name": "Player 1", "opponent": "Player 2","date":"Date"},inplace=True)

matches_df['Tourn'] = matches_df['event'].str.upper()

matches_df['score'] =matches_df['score'].str.replace(' V ',':')

#Clean tournament name
tourn_mappings = [
    ["GRAND SLAM OF DARTS","GRAND SLAM"]
]
for m in tourn_mappings:
    matches_df[matches_df['Tourn'] == m[0]]['Tourn'] = m[1]




c:\users\oliver.cairns\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [97]:
matches_df['Tourn'] = matches_df['Tourn'].str.replace('GRAND SLAM OF DARTS','GRAND SLAM')
#Only keep relevant columns
elo_df = matches_df.loc[:,['Player 1','Player 2','Outcome','Date','Tourn','score']]


In [98]:
#Drop mirror image fixtures. Quite crude, need to check
already = []
for player_name in elo_df['Player 1'].unique():
    #Keep if - player name not current one, or rival not in 'already' group. 
    elo_df = elo_df[(elo_df['Player 1'] != player_name) | (-elo_df['Player 2'].isin(already))]
    already.append(player_name)



#Keep surname only
elo_df['Surname 1'] = elo_df['Player 1'].str.split(' ',n=1).str.get(1).str.upper().str.strip()
elo_df['Surname 2'] = elo_df['Player 2'].str.split(' ',n=1).str.get(1).str.upper().str.strip()


In [99]:
#Turn date into datetime variable
elo_df['Date'] = pd.to_datetime(elo_df['Date'])

elo_df['year'] =elo_df['Date'].dt.year


In [71]:
#Merge on odds data - quite crude.

odds_df = pd.read_excel("../../Data/oddschecker/Darts_odds.xlsx")

def convert_frac_to_prob(frac):
    if "/" in frac:
        num, denom = frac.split("/")
        return int(denom) / (int(num) + int(denom))

odds_df['p1_prob'] = odds_df['odds_1'].apply(convert_frac_to_prob)
odds_df['p2_prob'] = odds_df['odds_2'].apply(convert_frac_to_prob)



In [72]:
#clean player names
odds_df['players'] = odds_df['players'].str.replace("O' ",'O ')


odds_df['Player 1'] = odds_df['players'].str.split(' - ',n=1).str.get(0)
odds_df['Player 2'] = odds_df['players'].str.split(' - ',n=1).str.get(1)

odds_df['Surname 1'] = odds_df['Player 1'].str.slice(0,-2).str.upper().str.strip()
odds_df['Surname 2'] = odds_df['Player 2'].str.slice(0,-2).str.upper().str.strip()

odds_df['Tourn']=odds_df['tournament_name'].str.split('/').str.get(-1).str.replace("-"," ").str.upper()



In [73]:
#Same odds, the opposite way around
opp_df = odds_df.copy()

opp_df.rename(index=str, \
columns= {"odds_1": "odds_2", "odds_2": "odds_1","Player 1":"Player 2","Player 2":"Player 1","Surname 1":"Surname 2", \
"Surname 2":"Surname 1"},inplace=True)

#Doesn't work for games > 10.
opp_df['score']=opp_df['score'].apply(lambda x: x[::-1])

odds_df = odds_df.append(opp_df,ignore_index=True,sort=False)
        


In [100]:
merged_df = pd.merge(odds_df, elo_df,  how='outer',left_on=['Surname 1','Surname 2','Tourn','score'],right_on=['Surname 1','Surname 2','Tourn','score'],indicator=True)

In [101]:
merged_df['_merge'].value_counts()

right_only    75996
left_only     11437
both           3349
Name: _merge, dtype: int64

In [95]:
left = merged_df[merged_df['_merge'] == 'left_only']
right = merged_df[merged_df['_merge'] == 'right_only']

In [107]:
def count_non_matches(df1,df2,col):
    
    first = df1[-df1[col].isin(df2[col])][col].value_counts()
    second = df2[col].value_counts()
    print('first - non matches')
    print(first)
    print()
    print('second - all')
    print(second)
    
count_non_matches(odds_df,elo_df,'Tourn')



first - non matches
EUROPEAN TOUR 5             668
EUROPEAN TOUR 3             668
EUROPEAN TOUR 2             668
EUROPEAN TOUR 4             668
EUROPEAN TOUR 1             664
EUROPEAN TOUR 6             474
EUROPEAN TOUR 8             474
EUROPEAN TOUR 7             472
CHAMPIONSHIP LEAGUE         400
WORLD CUP TEAMS             326
EUROPEAN TOUR 9             280
WORLD CUP SINGLES           244
EUROPEAN TOUR 10            188
WORLD CHAMPIONSHIP WOMEN    180
WORLD SERIES FINALS         136
EUROPEAN TOUR 11             94
EUROPEAN TOUR 12             94
CHAMPIONS LEAGUE             60
TOKYO MASTERS                30
PLAYERS CHAMPIONSHIP 9       28
WORLD CUP DOUBLES            26
SINGAPORE MASTERS            14
PLAYERS CHAMPIONSHIP 7       14
PLAYERS CHAMPIONSHIP 8       12
PLAYERS CHAMPIONSHIP 3       12
PLAYERS CHAMPIONSHIP 4       12
PLAYERS CHAMPIONSHIP 6       10
PLAYERS CHAMPIONSHIP 1       10
PLAYERS CHAMPIONSHIP 2       10
PLAYERS CHAMPIONSHIP 10       8
PLAYERS CHAMPIONSHIP

In [108]:
elo_df[elo_df['Tourn'].str.match("EUROPEAN")]['Tourn'].value_counts()

EUROPEAN CHAMPIONSHIP              302
EUROPEAN DARTS CLASSIC              26
EUROPEAN DARTS OPEN                 19
EUROPEAN CHAMPIONSHIP QUALIFIER     13
EUROPEAN MASTERS                     8
Name: Tourn, dtype: int64

Index(['odds_1', 'odds_2', 'players', 'score', 'tournament_name', 'year',
       'p1_prob', 'p2_prob', 'Player 1', 'Player 2', 'Surname 1', 'Surname 2',
       'Tourn'],
      dtype='object')